In [1]:
import numpy as np
import mediapipe as mp
import cv2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import tensorflow as tf
from tensorflow import timestamp
from tensorflow.keras.models import load_model
import time


In [2]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
# mp_face = mp.solutions.
hands = mp_hands.Hands(
                       max_num_hands=2,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)
pose = mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Drawing utility
mp_draw = mp.solutions.drawing_utils

In [4]:
"""
* Extracting raw landmark data from a specified video, then writing it to a json file
* the idea is that we will have to replicate the functionality in java script so the client can
* send the json data to the server.
*
* note: might consider uploading the npz file instead but have to test for that
"""
import json

# Reading From a video file
cap = cv2.VideoCapture('videos/SamplePushups.mp4')

cv2.namedWindow('MediaPipe', cv2.WINDOW_NORMAL)
cv2.resizeWindow('MediaPipe', 960, 540)  # width, height

# Data storage
frames_data = []
frame_index = 0
start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    try:
        # Convert to RGB format
        RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # --- Hands ---
        results = hands.process(RGB)

        frame_info = {
            "frame_index": frame_index,
            "timestamp": time.time() - start_time,
            "hands": {
                "Left": [{"x": 0, "y": 0, "z": 0}]*21,   # default zeros
                "Right": [{"x": 0, "y": 0, "z": 0}]*21
            },
            "pose": [{"x": 0, "y": 0, "z": 0}]*33  # default zeros
        }

        if results.multi_hand_landmarks:
            for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
                handedness = results.multi_handedness[i].classification[0].label if results.multi_handedness else "Unknown"
                hand_data = [{"x": lm.x, "y": lm.y, "z": lm.z} for lm in hand_landmarks.landmark]
                frame_info["hands"][handedness] = hand_data
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # --- Pose ---
        pose_results = pose.process(RGB)
        if pose_results.pose_landmarks:
            pose_data = [{"x": lm.x, "y": lm.y, "z": lm.z} for lm in pose_results.pose_landmarks.landmark]
            frame_info["pose"] = pose_data
            mp_draw.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        frames_data.append(frame_info)
        frame_index += 1

        cv2.imshow('MediaPipe', frame)
    except:
        break

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Save all frames to JSON file
session_data = {
    "metadata": {
        "source": "video",
        "fps": cap.get(cv2.CAP_PROP_FPS) or 30,  # fallback if FPS=0
        "num_frames": len(frames_data)
    },
    "frames": frames_data
}

with open("output/raw/hand_landmarks_session.json", "w") as f:
    json.dump(session_data, f, indent=2)

print("Data saved to hand_landmarks_session.json")

Data saved to hand_landmarks_session.json


In [7]:
import json
import numpy as np

# Load JSON file
with open("output/raw/hand_landmarks_session.json", "r") as f:
    session_data = json.load(f)

frames = session_data["frames"]
num_frames = len(frames)

# Each frame has:
# - left hand 21 × 3
# - right hand 21 × 3
# - pose 33 × 3
# Total features per frame: 21*3*2 + 33*3 = 225

all_landmarks = []

for frame in frames:
    frame_features = []

    # Hands
    hands_data = frame.get("hands", [])

    # Left hand
    left_hand = hands_data.get("left", [])
    if left_hand:
        for lm in left_hand["landmarks"]:
            frame_features += [lm["x"], lm["y"], lm["z"]]
    else:
        frame_features += [0.0] * 63  # 21*3 zeros if missing

    # Right hand
    right_hand = hands_data.get("right", [])
    if right_hand:
        for lm in right_hand["landmarks"]:
            frame_features += [lm["x"], lm["y"], lm["z"]]
    else:
        frame_features += [0.0] * 63

    # Pose
    pose_data = frame.get("pose", [])
    if pose_data:
        for lm in pose_data:
            frame_features += [lm["x"], lm["y"], lm["z"]]
    else:
        frame_features += [0.0] * 99  # 33*3 zeros if missing

    all_landmarks.append(frame_features)

# Convert to NumPy array
landmarks_array = np.array(all_landmarks, dtype=np.float32)

# Save as compressed npz
np.savez_compressed("output/compressed/landmarks.npz", landmarks=landmarks_array)

print(f"Saved {landmarks_array.shape} landmarks to landmarks.npz")


Saved (325, 225) landmarks to landmarks.npz


In [8]:
import numpy as np
import pandas as pd

def peek_landmarks_npz(npz_file):
    """
    Load a .npz file and return a pandas DataFrame for inspection.
    """
    data = np.load(npz_file)
    landmarks = data["landmarks"]

    # Generate column names
    columns = []

    # Left hand (21 × 3)
    for i in range(21):
        columns += [f"left_x{i}", f"left_y{i}", f"left_z{i}"]

    # Right hand (21 × 3)
    for i in range(21):
        columns += [f"right_x{i}", f"right_y{i}", f"right_z{i}"]

    # Pose (33 × 3)
    for i in range(33):
        columns += [f"pose_x{i}", f"pose_y{i}", f"pose_z{i}"]

    df = pd.DataFrame(landmarks, columns=columns)
    return df

# Example usage
df_peek = peek_landmarks_npz("output/compressed/landmarks.npz")
print(df_peek.head())


   left_x0  left_y0  left_z0  left_x1  left_y1  left_z1  left_x2  left_y2  \
0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
2      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
3      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
4      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   

   left_z2  left_x3  ...  pose_z29  pose_x30  pose_y30  pose_z30  pose_x31  \
0      0.0      0.0  ...  0.556724  0.321450  0.581000  0.410814  0.364779   
1      0.0      0.0  ...  0.532637  0.320817  0.580730  0.385674  0.364723   
2      0.0      0.0  ...  0.514099  0.320186  0.580121  0.368876  0.364711   
3      0.0      0.0  ...  0.569956  0.320000  0.578825  0.382927  0.364465   
4      0.0      0.0  ...  0.551992  0.319969  0.578101  0.386637  0.364468   

   pose_y31  pose_z31  pose_x32  pose_y32  pose_z32  
0  0.651570  0

In [9]:
df_peek.describe()

,left_x0,left_y0,left_z0,left_x1,left_y1,left_z1,left_x2,left_y2,left_z2,left_x3,...,pose_z29,pose_x30,pose_y30,pose_z30,pose_x31,pose_y31,pose_z31,pose_x32,pose_y32,pose_z32
count,325.0,325.0,325.0,325.0,325.0,325.0,325.0,325.0,325.0,325.0,...,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000,325.000000
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.584080,0.213824,0.535444,0.333522,0.271926,0.612753,0.565808,0.238839,0.659064,0.276709
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.072712,0.111653,0.040623,0.100336,0.094740,0.035950,0.067477,0.104127,0.015508,0.109162
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.371824,0.085583,0.425013,0.079829,0.164222,0.516294,0.311789,0.128784,0.535798,0.013693
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.534705,0.101013,0.495805,0.258813,0.179551,0.579207,0.525794,0.136848,0.649288,0.185847
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.588680,0.127629,0.507956,0.344516,0.201779,0.592100,0.569252,0.140821,0.656548,0.291080
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.634991,0.326196,0.576650,0.412320,0.366365,0.648566,0.611109,0.344807,0.672332,0.367235
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.767928,0.342089,0.582778,0.579742,0.378230,0.662463,0.771414,0.352258,0.675038,0.544647


In [10]:
df_peek.columns

Index(['left_x0', 'left_y0', 'left_z0', 'left_x1', 'left_y1', 'left_z1',
       'left_x2', 'left_y2', 'left_z2', 'left_x3',
       ...
       'pose_z29', 'pose_x30', 'pose_y30', 'pose_z30', 'pose_x31', 'pose_y31',
       'pose_z31', 'pose_x32', 'pose_y32', 'pose_z32'],
      dtype='object', length=225)